In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.ndimage import median_filter

from database_tools.processing.modify import bandpass
from database_tools.processing.utils import resample_signal

def _clean_frame(sig, thresh):
    """Set points too far from median value to median value."""
    sig = sig.reshape(-1) # must be 1d
    med = np.median(sig)
    mask = np.where( (sig > (med + thresh)) | (sig < (med - thresh)) )
    sig[mask] = med
    return sig

def _flip_signal(sig):
    """Flip signal data but subtracting the maximum value."""
    flipped = np.max(sig) - sig
    return flipped

In [ ]:
with open('bpm-frame.pkl', 'rb') as f:
    data = pkl.load(f)

red_frame = data['red_frame']
ir_frame = data['ir_frame']

In [ ]:
thresh = 2000
low = 0.5
high = 8.0
size = 5
fs = 200
fs_new = 125

In [ ]:
red_frame = np.array(red_frame, dtype=np.float32)
red_frame[np.isnan(red_frame)] = 0
ir_frame = np.array(ir_frame, dtype=np.float32)
ir_frame[np.isnan(ir_frame)] = 0

red_clean = _clean_frame(red_frame, thresh=thresh)
ir_clean = _clean_frame(ir_frame, thresh=thresh)

red_resamp = resample_signal(red_clean, fs_old=fs, fs_new=fs_new)
ir_resamp = resample_signal(ir_clean, fs_old=fs, fs_new=fs_new)

red_flip = _flip_signal(red_resamp)
ir_flip = _flip_signal(ir_resamp)

red_filt = bandpass(red_flip, low=low, high=high, fs=fs, method='butter')
ir_filt = bandpass(ir_flip, low=low, high=high, fs=fs, method='butter')

In [ ]:
fig = make_subplots(rows=2, cols=1)
fig.add_scatter(y=red_frame, row=1, col=1, name='red_frame')
fig.add_scatter(y=ir_frame, row=2, col=1, name='ir_frame')

In [ ]:
fig = make_subplots(rows=2, cols=1)
fig.add_scatter(y=red_resamp, row=1, col=1, name='red_resamp')
fig.add_scatter(y=ir_resamp, row=2, col=1, name='ir_resamp')

In [ ]:
fig = make_subplots(rows=2, cols=1)
fig.add_scatter(y=red_clean, row=1, col=1, name='red_clean')
fig.add_scatter(y=ir_clean, row=2, col=1, name='ir_clean')

In [ ]:
fig = make_subplots(rows=2, cols=1)
fig.add_scatter(y=red_flip, row=1, col=1, name='red_flip')
fig.add_scatter(y=ir_flip, row=2, col=1, name='ir_flip')

In [ ]:
fig = go.Figure()
fig.add_scatter(y=red_filt, name='red_filt')
fig.add_scatter(y=ir_filt, name='ir_filt')